In [1]:
import tensorflow as tf
from dataset_tf import VoxDataset, DataGenerator
from tqdm import tqdm
import numpy as np
from weighted_mse import WeightedLoss
import matplotlib.pyplot as plt
import os
import pickle

In [2]:
models = os.listdir("./models")

In [3]:
models

['e1_w_ones_w_att_w_weightedMSE_w_customL_17l_lr00001_male',
 '.DS_Store',
 'e1_w_ones_w_weightedMSE_w_customL2_14l_lr00001_male_face_only',
 'e1_w_ones_w_weightedMSE_w_customL2_16l_lr00001_male_hausdorff',
 'e1_w_ones_w_weightedMSE_w_customL2_12l_lr00001_male_directed_hausdorff',
 'e1_w_ones_w_att_w_weightedMSE_16l_lr00001_male',
 'e1_w_ones_w_weightedMSE_w_customL2_10l_lr00001_male_hausdorff',
 'e1_w_ones_w_weightedMSE_w_customL_15l_lr00001_male',
 'e1_w_ones_w_weightedMSE_w_customL2_14l_lr00001_male_hausdorff',
 'e1_w_ones_w_weightedMSE_w_customL2_10l_lr00001_male_directed_hausdorff']

In [4]:
batch = 32
loss_weight = 1
model_name = None

In [5]:
test = VoxDataset(gender="m")

In [6]:
test = DataGenerator(test, batch_size=batch)

In [7]:
base_loss = tf.keras.losses.MeanSquaredError()

In [8]:
attention_ids = np.load("./weight_indices.npy")

In [9]:
landmark_indices = {
      "nose_bridge": (3516, 3526),
      "r_eye_lid": (3690, 2265),
      "l_eye_lid": (3856, 809),
      "lip": (3543, 3503),
      "r_lip_bend": (2850, 3798),
      "l_lip_bend": (1735, 3021),
      "r_lip_jaw": (3798, 3406),
      "l_lip_jaw": (3021, 3614),
      "lip_chin": (3503, 3487),
      "orbital_lower": (3710, 3866),
      "oribtal_upper": (3154, 2135),
      "puffer": (3436, 3667),
      "mouth_corner": (2827, 1710),
      "jaw_end": (3406, 3614),
      "upper_neck": (3451, 3693),
      "lower_neck": (3264, 3363),
      "ear_end": (856, 288)
  }

In [10]:
metrics = {}

In [11]:
summary = {}

In [12]:
for model_name in models:
    if model_name == '.DS_Store':
        continue

    with open(f'./models/{model_name}/loss_config.pkl', 'rb') as file:
        custom_metric = pickle.load(file)

    weighted_loss = WeightedLoss(base_loss=base_loss, attention_ids=attention_ids,custom_metric_dict=custom_metric, weight=loss_weight, batch_size=batch)
    model = tf.keras.models.load_model(f"./models/{model_name}/model.h5", custom_objects={"WeightedLoss" : weighted_loss})

    summary[model_name] = model.summary()
    y_pred = []
    for i in tqdm(test):
        pred = (model.predict(i[0], verbose=0)
                *201.41335
                )
        for j in range(len(pred)):
            y_pred.append([VoxDataset.to_mesh_points(i[1][j]), VoxDataset.to_mesh_points(pred[j])])

    
        

    distance_diffs = {
        "nose_bridge": 0,
        "r_eye_lid": 0,
        "l_eye_lid": 0,
        "lip": 0,
        "r_lip_bend": 0,
        "l_lip_bend": 0,
        "r_lip_jaw": 0,
        "l_lip_jaw": 0,
        "lip_chin": 0,
        "orbital_lower": 0,
        "oribtal_upper": 0,
        "puffer": 0,
        "mouth_corner": 0,
        "jaw_end": 0,
        "upper_neck": 0,
        "lower_neck": 0,
        "ear_end": 0
    }
    for i in y_pred:
        for landmark, indices in landmark_indices.items():
            ground_truth_dist = np.abs(np.linalg.norm(i[0][indices[0]] - i[0][indices[1]])) / np.abs(np.linalg.norm(i[0][856] - i[0][288]))
            predicted_dist = np.abs(np.linalg.norm(i[1][indices[0]] - i[1][indices[1]])) / np.abs(np.linalg.norm(i[1][856] - i[1][288]))
            distance_diffs[landmark] = np.abs(distance_diffs[landmark] + (ground_truth_dist - predicted_dist))

        mse = (1 / len(y_pred)) * sum((i[0][j] - i[1][j])**2 for j in range(i[0].shape[0]))
        distance_diffs['mse'] = mse

        mae = (1 / len(y_pred)) * sum(np.abs(i[0][j] - i[1][j]) for j in range(i[0].shape[0]))
        distance_diffs['mae'] = mae

    for landmark, diffs in distance_diffs.items():
        if landmark == "mse":
            print(diffs)
            distance_diffs[landmark] = np.mean(diffs)
        elif landmark == "mae":
            print(diffs)
            distance_diffs[landmark] = np.mean(diffs)
        else:
            distance_diffs[landmark] = diffs / len(y_pred)

    metrics[model_name] = distance_diffs

    


    

2024-05-04 17:32:37.624984: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-05-04 17:32:37.625007: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-05-04 17:32:37.625010: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-05-04 17:32:37.625040: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-04 17:32:37.625057: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 384)]                0         []                            
                                                                                                  
 dense (Dense)               (None, 384)                  147840    ['input_1[0][0]']             
                                                                                                  
 attention (Attention)       (None, 384)                  0         ['dense[0][0]',               
                                                                     'dense[0][0]']               
                                                                                                  
 multiply (Multiply)         (None, 384)                  0         ['attention[0][0]',       

100%|██████████| 2100/2100 [04:23<00:00,  7.96it/s]


[187.79396 158.00049 191.8732 ]
[3.2156818 2.4543388 3.3043647]
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 384)]             0         
                                                                 
 dense_26 (Dense)            (None, 384)               147840    
                                                                 
 dense_27 (Dense)            (None, 384)               147840    
                                                                 
 dense_28 (Dense)            (None, 500)               192500    
                                                                 
 dense_29 (Dense)            (None, 768)               384768    
                                                                 
 dense_30 (Dense)            (None, 1000)              769000    
                                                             

100%|██████████| 2100/2100 [04:02<00:00,  8.67it/s]


...

[190.57649 162.81152 192.79753]
[3.2336183 2.514633  3.3063571]
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 384)]             0         
                                                                 
 dense_13 (Dense)            (None, 384)               147840    
                                                                 
 dense_14 (Dense)            (None, 384)               147840    
                                                                 
 dense_15 (Dense)            (None, 500)               192500    
                                                                 
 dense_16 (Dense)            (None, 768)               384768    
                                                                 
 dense_17 (Dense)            (None, 1000)              769000    
                                                             

100%|██████████| 2100/2100 [04:05<00:00,  8.55it/s]


[196.49103 169.25737 193.9745 ]
[3.290285  2.5335703 3.317637 ]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 384)]             0         
                                                                 
 dense (Dense)               (None, 384)               147840    
                                                                 
 dense_1 (Dense)             (None, 384)               147840    
                                                                 
 dense_2 (Dense)             (None, 500)               192500    
                                                                 
 dense_3 (Dense)             (None, 768)               384768    
                                                                 
 dense_4 (Dense)             (None, 1000)              769000    
                                                               

100%|██████████| 2100/2100 [03:16<00:00, 10.71it/s]


[195.88094 162.2221  196.56644]
[3.2838295 2.5068223 3.3417065]


In [13]:
metrics

{'e1_w_ones_w_att_w_weightedMSE_w_customL_17l_lr00001_male': {'nose_bridge': 9.57991961123688e-06,
  'r_eye_lid': 1.5005493603114572e-05,
  'l_eye_lid': 5.886934176530867e-06,
  'lip': 6.4621224356371734e-06,
  'r_lip_bend': 4.704182169267109e-06,
  'l_lip_bend': 0.4400407269650272,
  'r_lip_jaw': 0.00010725340672901699,
  'l_lip_jaw': 0.0006819205624716623,
  'lip_chin': 9.154597918192546e-05,
  'orbital_lower': 1.4611386827060154e-07,
  'oribtal_upper': 4.6893787969435965e-06,
  'puffer': 0.007812768215136158,
  'mouth_corner': 1.6104738183674359e-06,
  'jaw_end': 0.00047524883633568175,
  'upper_neck': 0.0011583995819091798,
  'lower_neck': 0.0005357798508235387,
  'ear_end': 0.0,
  'mse': 179.22253,
  'mae': 2.9914618},
 'e1_w_ones_w_weightedMSE_w_customL2_14l_lr00001_male_face_only': {'nose_bridge': 1.0760567550148283e-07,
  'r_eye_lid': 0.002825551432047394,
  'l_eye_lid': 0.00209006086413172,
  'lip': 0.006003539398467789,
  'r_lip_bend': 0.0037529321707829477,
  'l_lip_bend': 0

In [14]:
for name, values in metrics.items():
    mean_list = []
    for val_name, val in values.items():
        if val_name == "mse" or val_name == "mae":
            continue
        mean_list.append(val)

    print(name, " : ", np.mean(np.square(mean_list)))
    metrics[name]['custom_mae'] = np.mean(np.abs(mean_list))

e1_w_ones_w_att_w_weightedMSE_w_customL_17l_lr00001_male  :  0.011394071816817163
e1_w_ones_w_weightedMSE_w_customL2_14l_lr00001_male_face_only  :  5.880426748706014e-06
e1_w_ones_w_weightedMSE_w_customL2_16l_lr00001_male_hausdorff  :  0.14358552473918118
e1_w_ones_w_weightedMSE_w_customL2_12l_lr00001_male_directed_hausdorff  :  3.5996165973555584e-06
e1_w_ones_w_att_w_weightedMSE_16l_lr00001_male  :  4.474092140009594e-06
e1_w_ones_w_weightedMSE_w_customL2_10l_lr00001_male_hausdorff  :  1.2441908904390758e-05
e1_w_ones_w_weightedMSE_w_customL_15l_lr00001_male  :  1.312863111346723e-05
e1_w_ones_w_weightedMSE_w_customL2_14l_lr00001_male_hausdorff  :  0.0001313624125927746
e1_w_ones_w_weightedMSE_w_customL2_10l_lr00001_male_directed_hausdorff  :  1.802058419169845e-06


In [15]:
metrics

{'e1_w_ones_w_att_w_weightedMSE_w_customL_17l_lr00001_male': {'nose_bridge': 9.57991961123688e-06,
  'r_eye_lid': 1.5005493603114572e-05,
  'l_eye_lid': 5.886934176530867e-06,
  'lip': 6.4621224356371734e-06,
  'r_lip_bend': 4.704182169267109e-06,
  'l_lip_bend': 0.4400407269650272,
  'r_lip_jaw': 0.00010725340672901699,
  'l_lip_jaw': 0.0006819205624716623,
  'lip_chin': 9.154597918192546e-05,
  'orbital_lower': 1.4611386827060154e-07,
  'oribtal_upper': 4.6893787969435965e-06,
  'puffer': 0.007812768215136158,
  'mouth_corner': 1.6104738183674359e-06,
  'jaw_end': 0.00047524883633568175,
  'upper_neck': 0.0011583995819091798,
  'lower_neck': 0.0005357798508235387,
  'ear_end': 0.0,
  'mse': 179.22253,
  'mae': 2.9914618,
  'custom_mae': 0.02652657223624081},
 'e1_w_ones_w_weightedMSE_w_customL2_14l_lr00001_male_face_only': {'nose_bridge': 1.0760567550148283e-07,
  'r_eye_lid': 0.002825551432047394,
  'l_eye_lid': 0.00209006086413172,
  'lip': 0.006003539398467789,
  'r_lip_bend': 0.0

In [16]:
np.linalg.norm([1,1], [2,2])

array([1.41421356, 1.41421356])

In [17]:
type(model.summary()[0])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 384)]             0         
                                                                 
 dense (Dense)               (None, 384)               147840    
                                                                 
 dense_1 (Dense)             (None, 384)               147840    
                                                                 
 dense_2 (Dense)             (None, 500)               192500    
                                                                 
 dense_3 (Dense)             (None, 768)               384768    
                                                                 
 dense_4 (Dense)             (None, 1000)              769000    
                                                                 
 dense_5 (Dense)             (None, 1536)              153753

TypeError: 'NoneType' object is not subscriptable

In [48]:
model_name = "e1_w_ones_w_weightedMSE_w_customL2_10l_lr00001_male_hausdorff"

In [49]:
with open(f'./models/{model_name}/loss_config.pkl', 'rb') as file:
    custom_metric = pickle.load(file)

weighted_loss = WeightedLoss(base_loss=base_loss, attention_ids=attention_ids,custom_metric_dict=custom_metric, weight=loss_weight, batch_size=batch)
model = tf.keras.models.load_model(f"./models/{model_name}/model.h5", custom_objects={"WeightedLoss" : weighted_loss})

In [50]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 384)]             0         
                                                                 
 dense (Dense)               (None, 384)               147840    
                                                                 
 dense_1 (Dense)             (None, 384)               147840    
                                                                 
 dense_2 (Dense)             (None, 500)               192500    
                                                                 
 dense_3 (Dense)             (None, 768)               384768    
                                                                 
 dense_4 (Dense)             (None, 1000)              769000    
                                                                 
 dense_5 (Dense)             (None, 1536)              153753